In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors


# Chargement des fichiers
bikes_df = pd.read_csv('/content/bikes.csv', delimiter=';')
bikeshops_df = pd.read_csv('/content/bikeshops.csv', delimiter=';')
orders_df = pd.read_csv('/content/orders.csv', delimiter=';')
customers_df = pd.read_csv('/content/Customers.csv', delimiter=',')

In [ ]:
# Jointure entre les commandes (orders) et les vélos (bikes) sur 'product.id' et 'bike.id'
orders_bikes_df = pd.merge(orders_df, bikes_df, left_on='product.id', right_on='bike.id')


In [ ]:
orders_bikes_df['ID'] = range(1, len(orders_bikes_df) + 1)
orders_bikes_df= orders_bikes_df.reset_index(drop=True)
orders_bikes_df = orders_bikes_df.set_index('ID')
orders_bikes_df = orders_bikes_df.drop(columns=['Unnamed: 0'])



In [ ]:
# Jointure entre les commandes (orders) et  les clients (customers) sur 'product.id' et 'bike.id'
orders_customers_df = pd.merge(orders_df, customers_df, left_on='customer.id', right_on='CustomerKey', how='left')


In [ ]:
# Jointure entre les deux tables résultantes sur 'order.id', 'order.line','order.date','product.id
df_merged = pd.merge(orders_customers_df, orders_bikes_df, on=['order.id', 'order.line','order.date','product.id'], how='inner')

In [ ]:
df_merged = df_merged.T.drop_duplicates().T
df_merged = df_merged.drop(columns=['Unnamed: 0'])
df=df_merged

Traitement des valeurs manquantes

In [ ]:
data = df

# Vérification des valeurs manquantes dans le dataset
missing_values_summary = data.isnull().sum()

# Création d'un DataFrame pour afficher les colonnes avec des valeurs manquantes et leur pourcentage
missing_values_df = pd.DataFrame({
    'Column': missing_values_summary.index,
    'Missing Values': missing_values_summary.values,
    'Percentage': (missing_values_summary.values / len(data)) * 100
})

# Filtrage pour afficher seulement les colonnes ayant des valeurs manquantes
missing_values_filtered = missing_values_df[missing_values_df['Missing Values'] > 0]
print("Résumé des valeurs manquantes :")
print(missing_values_filtered)

# Identification des colonnes numériques et catégorielles
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = data.select_dtypes(include=['object']).columns

# Imputation des colonnes numériques (avec la moyenne) et des colonnes catégorielles (avec le mode)
for col in numeric_cols:
    if col in missing_values_filtered['Column'].values:
        data[col].fillna(data[col].mean(), inplace=True)

for col in categorical_cols:
    if col in missing_values_filtered['Column'].values:
        data[col].fillna(data[col].mode()[0], inplace=True)


Résumé des valeurs manquantes :
Empty DataFrame
Columns: [Column, Missing Values, Percentage]
Index: []


Traitement des doublons

In [ ]:
print(f"Nombre de doublons avant suppression : {data.duplicated().sum()}")


Nombre de doublons avant suppression : 0


Traitement des valeurs abberantes

In [ ]:
# Définition d'une fonction qui détecte les valeurs aberrantes en utilisant l'écart interquartile (IQR)
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # Premier quartile
    Q3 = df[column].quantile(0.75)  # Troisième quartile
    IQR = Q3 - Q1  # Écart interquartile

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identification des valeurs aberrantes
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    return outliers

# Vérification des valeurs aberrantes dans la colonne 'price'
price_outliers = detect_outliers(data, 'price')
print(f"Nombre de valeurs aberrantes dans 'price' : {len(price_outliers)}")

# Vérification des valeurs aberrantes dans la colonne 'quantity_x'
quantity_outliers = detect_outliers(data, 'quantity_x')
print(f"Nombre de valeurs aberrantes dans 'quantity_x' : {len(quantity_outliers)}")

# Filtrage des données aberrantes en excluant les valeurs en dehors des bornes
data_filtered = data[
    (data['price'] >= data['price'].quantile(0.25) - 1.5 * (data['price'].quantile(0.75) - data['price'].quantile(0.25))) &
    (data['price'] <= data['price'].quantile(0.75) + 1.5 * (data['price'].quantile(0.75) - data['price'].quantile(0.25))) &
    (data['quantity_x'] >= data['quantity_x'].quantile(0.25) - 1.5 * (data['quantity_x'].quantile(0.75) - data['quantity_x'].quantile(0.25))) &
    (data['quantity_x'] <= data['quantity_x'].quantile(0.75) + 1.5 * (data['quantity_x'].quantile(0.75) - data['quantity_x'].quantile(0.25)))
]

# Vérification de la taille du dataset après filtrage
print(f"Taille du dataset après filtrage : {data_filtered.shape}")

# Affichage des premières lignes du dataset après le filtrage des valeurs aberrantes
print("\nDataset après filtrage des valeurs aberrantes :")
print(data_filtered.head())
data=data_filtered


Nombre de valeurs aberrantes dans 'price' : 1376
Nombre de valeurs aberrantes dans 'quantity_x' : 2542
Taille du dataset après filtrage : (11935, 23)

Dataset après filtrage des valeurs aberrantes :
  order.id order.line order.date customer.id_x product.id quantity_x  Prefix  \
0        1          1   1/7/2011             2         48          1    Mrs.   
1        1          2   1/7/2011             2         52          1    Mrs.   
2        2          1  1/10/2011            10         76          1     Ms.   
3        2          2  1/10/2011            10         52          1     Ms.   
5        3          2  1/10/2011             6         50          1    Miss   

   FirstName  LastName    BirthDate  ...  AnnualIncome  TotalChildren  \
0      Alice     Smith   02/12/1988  ...         60000              2   
1      Alice     Smith   02/12/1988  ...         60000              2   
2      Fiona      Teal   12/09/1989  ...         46000              1   
3      Fiona      Teal   12/

Encodage des variables catégorielles :

In [ ]:
# Liste des colonnes catégorielles pertinentes
categorical_cols_to_encode = ['category1', 'category2', ' Occupation', ' HomeOwner',
                              ' EducationLevel', ' MaritalStatus', ' Gender', 'frame']

# Encodage de fréquence pour chaque colonne catégorielle pertinente
for col in categorical_cols_to_encode:
    if col in data.columns:
        # Remplacer chaque catégorie par la fréquence d'apparition de la catégorie
        freq_encoding = data[col].value_counts(normalize=True)
        data[col] = data[col].map(freq_encoding)

# Affichage des premières lignes du dataset après encodage
print("\nDataset après encodage de fréquence :")
print(data.head())

# Affichage de la taille du dataset après encodage
print(f"\nTaille du dataset après encodage : {data.shape}")


Dataset après encodage de fréquence :
  order.id order.line order.date customer.id_x product.id quantity_x  Prefix  \
0        1          1   1/7/2011             2         48          1    Mrs.   
1        1          2   1/7/2011             2         52          1    Mrs.   
2        2          1  1/10/2011            10         76          1     Ms.   
3        2          2  1/10/2011            10         52          1     Ms.   
5        3          2  1/10/2011             6         50          1    Miss   

   FirstName  LastName    BirthDate  ...   AnnualIncome   TotalChildren  \
0      Alice     Smith   02/12/1988  ...          60000               2   
1      Alice     Smith   02/12/1988  ...          60000               2   
2      Fiona      Teal   12/09/1989  ...          46000               1   
3      Fiona      Teal   12/09/1989  ...          46000               1   
5       Jane      Blue   14/04/1995  ...          32000               0   

   EducationLevel  Occupation

<ipython-input-10-e1e69120d802>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].map(freq_encoding)
<ipython-input-10-e1e69120d802>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].map(freq_encoding)
<ipython-input-10-e1e69120d802>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [ ]:
# Conversion la colonne 'order.date' en format datetime
data['order.date'] = pd.to_datetime(data['order.date'])

# Création d'une matrice client-produit basée sur l'historique des achats
# La matrice contient les clients en lignes et les produits en colonnes avec les quantités achetées comme valeurs
purchase_matrix = data.pivot_table(index='customer.id_x', columns='product.id', values='quantity_x', aggfunc='sum', fill_value=0)

# Visualisation un échantillon de la matrice client-produit (facultatif)
print("Aperçu de la matrice client-produit :")
print(purchase_matrix.head())

# Définition le modèle KNN pour trouver les clients les plus similaires
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(purchase_matrix)

# Fonction pour recommander des produits à un client spécifique
def recommend_products(customer_id, n_neighbors=5, n_recommendations=5):
    # les clients les plus proches
    distances, indices = knn.kneighbors(purchase_matrix.loc[[customer_id]], n_neighbors=n_neighbors+1)

    # Exclure le client lui-même en prenant les autres voisins
    similar_customers = indices.flatten()[1:]

    # Agrégation les achats des clients similaires
    similar_purchases = purchase_matrix.iloc[similar_customers].mean(axis=0)

    # Filtrage les produits que le client n'a pas encore achetés
    customer_purchases = purchase_matrix.loc[customer_id]
    recommendations = similar_purchases[customer_purchases == 0]

    # Trie les recommandations en fonction des scores moyens d'achat
    recommended_products = recommendations.sort_values(ascending=False).head(n_recommendations)
    return recommended_products

# Exemple de recommandation pour un client (par exemple, client avec ID 1)
recommendations = recommend_products(customer_id=1, n_neighbors=5, n_recommendations=5)
print("\nProduits recommandés pour le client 1 :")
print(recommendations)


Aperçu de la matrice client-produit :
product.id     4   5   6   7   8   9   10  11  12  13  ...  88  89  90  91  \
customer.id_x                                          ...                   
1               5   6   2   3   1   1   0   2   3   2  ...   3   3   3   1   
2               8   7  10   7   0   0   1   3   6  10  ...   6   9  14  12   
3               4   3   1   5   0   2   4   4   5   7  ...   1   1   1   2   
4               2   1   1   3   1   4   1   3   3   4  ...   5   5   4   6   
5               4   5   2   5   1   3   6   6   4   9  ...   0   0   1   1   

product.id     92  93  94  95  96  97  
customer.id_x                          
1               3   6   5   1   0   1  
2               8   5   7   6   6   7  
3               0   0   0   1   0   1  
4               4   8   3   4   3   3  
5               1   0   0   0   0   0  

[5 rows x 86 columns]

Produits recommandés pour le client 1 :
product.id
96    17.6
21     9.8
42     9.2
35     9.2
20     8.2
dtype

<ipython-input-12-b7a00ae37f26>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['order.date'] = pd.to_datetime(data['order.date'])
